load and save a balanced background and eval set. background from the training set and eval from the testing set

In [10]:
SEED = 42
QUICK_DEBUG = False
MEDIUM_DEBUG = True
RANDOM_STATE = SEED
OUTPUT_FOLDER_NAME = "final_datasets/shap"
BACKGROUND_SIZE = 256

In [11]:
import copy
import numpy as np
from final_pipeline import ModelWrapper, get_data, setup_device
from full_pipeline import find_project_root
import matplotlib.pyplot as plt
import numpy as np
from explain_model_helpers import ShapModel, get_config, get_patient_predictions, get_pred_threshold
from preprocess import over_under_sample
import os
import json

In [12]:
project_root = find_project_root()
data_output_folder = f"{project_root}/dataset/{OUTPUT_FOLDER_NAME}"
if not os.path.exists(data_output_folder):
    os.makedirs(data_output_folder)

In [13]:
project_root = find_project_root()
results_name = "medium_model_no_sampling"
config = get_config(project_root, results_name)

device = setup_device()
train_data = get_data(config, "train")
val_data = get_data(config, "val")
test_data = get_data(config, "test")
in_dim = train_data.X.shape[1]

Using MPS device


/Users/aidend/Developer/ds_uni/dl-sepsis-prediction/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/Users/aidend/Developer/ds_uni/dl-sepsis-prediction/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower th

In [14]:
background_df = copy.deepcopy(train_data.df)

background_patient_df = background_df.groupby("patient_id")["SepsisLabel"].max().reset_index()

# Count patients in each group.
counts = background_patient_df["SepsisLabel"].value_counts()

neg_patients = background_patient_df[background_patient_df["SepsisLabel"] == 0]
pos_patients = background_patient_df[background_patient_df["SepsisLabel"] == 1]
sampled_neg_patients = neg_patients.sample(n=BACKGROUND_SIZE // 2, replace=False, random_state=SEED)
sampled_pos_patients = pos_patients.sample(n=BACKGROUND_SIZE // 2, replace=False, random_state=SEED)

background_df = background_df[background_df["patient_id"].isin(sampled_neg_patients["patient_id"]) | background_df["patient_id"].isin(sampled_pos_patients["patient_id"])]

print(background_df.shape)
print(background_df.groupby("patient_id")["SepsisLabel"].max().value_counts())

background_df.to_parquet(f"{data_output_folder}/background.parquet")

# save the patient ids as json
patient_ids = background_df["patient_id"].unique()
with open(f"{data_output_folder}/background_patient_ids.json", "w") as f:
    json.dump(patient_ids.tolist(), f)


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
(9611, 109)
SepsisLabel
1    100
0    100
Name: count, dtype: int64


In [15]:
eval_df = copy.deepcopy(test_data.df)
eval_df = over_under_sample(eval_df, method="undersample", minority_ratio=0.5, random_state=SEED)
eval_df.to_parquet(f"{data_output_folder}/eval.parquet")

patient_ids = eval_df["patient_id"].unique()
with open(f"{data_output_folder}/eval_patient_ids.json", "w") as f:
    json.dump(patient_ids.tolist(), f)

Patient-level balance statistics:
Total patients: 1172
Label 1: 586 patients (50.00%)
Label 0: 586 patients (50.00%)
Imbalance ratio (majority/minority): 1.00
(57226, 109)
SepsisLabel
1    586
0    586
Name: count, dtype: int64


In [16]:
background_data = get_data(config, "shap_background")
eval_data = get_data(config, "shap_eval")

/Users/aidend/Developer/ds_uni/dl-sepsis-prediction/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/Users/aidend/Developer/ds_uni/dl-sepsis-prediction/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower th

In [17]:
shap_output_folder = f"{project_root}/models/model_B/results/{config['xperiment']['name']}/shap"

In [18]:
model = ModelWrapper(config, device, in_dim)
model.load_saved_weights()

/Users/aidend/Developer/ds_uni/dl-sepsis-prediction/.venv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [19]:
shap_model = ShapModel(model.model, background_data, device, pad_value=0.0)
shap_vals, masks = shap_model.get_shap_values(eval_data)  # (256, 400, 107)
shap_model.save(shap_output_folder)

SHAP batches:   0%|          | 0/127 [00:03<?, ?it/s]


KeyboardInterrupt: 